<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random

In [2]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v0' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# print out the observation and action spaces
print(env.action_space.n) # number of valid actions we can take 
print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1



2
4


In [34]:
# hyperparameters, according to section 4.1 Parameter Settings
deltaThreshold = 3.0

c1 = c2 = 1.0
c3 = 0.4

numUntilStagnant = 15
genomeMutationChance = 0.8
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.001
smallPopulationNewNodeChance = 0.03
smallPopulationNewConnectionChance = 0.05
largePopulationNewConnectionChance = 0.3

In [33]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [20]:
class Genome:
    # initialize class variables 
    def __init__(self):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.species = 0
        
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        
    def initRandomGenome(self):
        for i in range(inputs):
            self.insertNode('input')
        for i in range(outputs):
            self.insertNode('output')
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(node, otherNode, random.random())
            
        
    def initGenome(self, nodeDict, connectDict):
        self.nodeGenes = nodeDict
        self.connectGens = connectDict
        
             
    def insertNode(self, nodeType='input', value = '0.0'):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = '0.5', isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes): # check that the connection doesnt already exist 
                self.connectGenes[(outNode, inNode)] = [self.globalInnovation, weight, isExpressed]
                self.globalInnovation += 1
            else: 
                print('Connection already exists. Did not insert')
        else:
            print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) not in self.connectGenes:
                self.connectGenes[(outNode, inNode)] = [self.globalInnovation, random.random(), True]
                self.globalInnovation += 1
            else:
                print('Error! nodes already connected')
        else:
            print('Could not find node(s). Did not mutate')
            
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
            
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        return env.action_space.sample()
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

In [9]:
genome=Genome()
genome.initRandomGenome()


In [10]:
genome.printNodeGenes()
genome.printActiveConnectGenes()

{1: ['input', '0.0'], 2: ['input', '0.0'], 3: ['input', '0.0'], 4: ['input', '0.0'], 5: ['input', '0.0'], 6: ['output', '0.0'], 7: ['output', '0.0']}
(1, 6) 0.295853110322872
(1, 7) 0.43997264686223114
(2, 6) 0.16973338641236457
(2, 7) 0.18630857515300991
(3, 6) 0.19065053698494494
(3, 7) 0.21567729477697573
(4, 6) 0.6461327149224014
(4, 7) 0.5641040927550108
(5, 6) 0.8065439132871849
(5, 7) 0.3081420763091025


In [50]:
genome1 = Genome()
genome1.initRandomGenome()
genome1.mutateAddConnection(1, 2)
genome1.mutateAddNode(1, 2)

genome1.printConnectGenes()

genome2 = Genome()
genome2.initRandomGenome()

genome2.printConnectGenes()

# TODO: Verify this works properly 
def delta(genome1,genome2, c1=0.1, c2=0.1, c3=0.1):
    connectionMismatch = []
    for connection1 in genome1.connectGenes:
        if (connection1 not in genome2.connectGenes):
            connectionMismatch.append(connection1)
    for connection2 in genome2.connectGenes:
        if (connection2 not in genome1.connectGenes):
            connectionMismatch.append(connection2)
    diffsum = 0
    num = 0
    for connection1 in genome1.connectGenes:
        if (connection1 not in connectionMismatch):
            diffsum += abs(genome1.connectGenes[connection1][1] - genome2.connectGenes[connection1][1])
            num += 1
    averageWeightDifference = diffsum/num
    #print(averageWeightDifference)
    N = 0
    if len(genome1.connectGenes) >= len(genome2.connectGenes):
        N = len(genome1.connectGenes)
    else:
        N = len(genome2.connectGenes)
    #print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
    return ((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference)
    
delta(genome1=genome1,genome2=genome2, c1=c1, c2=c2, c3=c3)

{(1, 6): [1, 0.6142642317717515, True], (1, 7): [2, 0.17704433725475432, True], (2, 6): [3, 0.30488626931029694, True], (2, 7): [4, 0.5994561800373995, True], (3, 6): [5, 0.03665240710510942, True], (3, 7): [6, 0.5966580856592291, True], (4, 6): [7, 0.700207405160988, True], (4, 7): [8, 0.8007014027742866, True], (5, 6): [9, 0.12721716503246128, True], (5, 7): [10, 0.008031849017742343, True], (1, 2): [11, 0.724807581878696, False], (1, 8): [12, 1.0, True], (8, 2): [13, 0.724807581878696, True]}
{(1, 6): [1, 0.5805705222194604, True], (1, 7): [2, 0.9133221098163722, True], (2, 6): [3, 0.3247538130162717, True], (2, 7): [4, 0.03125241680228352, True], (3, 6): [5, 0.02504090812063242, True], (3, 7): [6, 0.05581903549830636, True], (4, 6): [7, 0.30996053762558984, True], (4, 7): [8, 0.9135376025331361, True], (5, 6): [9, 0.6074654916033674, True], (5, 7): [10, 0.11202676267174194, True]}


0.5814512473672437

In [11]:
# connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [54]:
max_generations = 20
population = 20

for generation in range(max_generations):
    species = []
    pop= []
    for genome in range(population):
        g = Genome()
        g.initRandomGenome()
        
        if (not species):
            species.append(g)
        else:
            for og in species:
                if (delta(g, og, c1, c2, c3) > deltaThreshold):
                    species.append(g)
                else:
                    g.species = species.index(og)
                    pop.append(g)
                    
                    
    print("Number of different species in generation {}: {}".format(generation, len(species)))


0
Number of different species in generation 0: 1
0
Number of different species in generation 1: 1
0
Number of different species in generation 2: 1
0
Number of different species in generation 3: 1
0
Number of different species in generation 4: 1
0
Number of different species in generation 5: 1
0
Number of different species in generation 6: 1
0
Number of different species in generation 7: 1
0
Number of different species in generation 8: 1
0
Number of different species in generation 9: 1
0
Number of different species in generation 10: 1
0
Number of different species in generation 11: 1
0
Number of different species in generation 12: 1
0
Number of different species in generation 13: 1
0
Number of different species in generation 14: 1
0
Number of different species in generation 15: 1
0
Number of different species in generation 16: 1
0
Number of different species in generation 17: 1
0
Number of different species in generation 18: 1
0
Number of different species in generation 19: 1


In [15]:
# testing it to run in gym 

for tries in range(20):
    observation = env.reset()
    genome=Genome()
    genome.initRandomGenome()
    t = 0
    while(True):
        env.render()
        action = genome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        genome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Genome fitness: {}".format(genome.fitness))
            break
        t += 1
            
env.close()

# for i_episode in range(20):
#     observation = env.reset()
#     for t in range(100):
#         env.render()
#         print(observation)
#         action = env.action_space.sample()
#         observation, reward, done, info = env.step(action)
#         if done:
#             print("Episode finished after {} timesteps".format(t+1))
#             break
# env.close()

Episode finished after 46 timesteps
Genome fitness: 46.0
Episode finished after 29 timesteps
Genome fitness: 29.0
Episode finished after 25 timesteps
Genome fitness: 25.0
Episode finished after 11 timesteps
Genome fitness: 11.0
Episode finished after 30 timesteps
Genome fitness: 30.0
Episode finished after 14 timesteps
Genome fitness: 14.0
Episode finished after 31 timesteps
Genome fitness: 31.0
Episode finished after 40 timesteps
Genome fitness: 40.0
Episode finished after 14 timesteps
Genome fitness: 14.0
Episode finished after 44 timesteps
Genome fitness: 44.0
Episode finished after 39 timesteps
Genome fitness: 39.0
Episode finished after 23 timesteps
Genome fitness: 23.0
Episode finished after 47 timesteps
Genome fitness: 47.0
Episode finished after 18 timesteps
Genome fitness: 18.0
Episode finished after 19 timesteps
Genome fitness: 19.0
Episode finished after 14 timesteps
Genome fitness: 14.0
Episode finished after 21 timesteps
Genome fitness: 21.0
Episode finished after 17 times